In [1]:
import pickle as pk
import argparse
import pickle as pk
import time
from re import S
import torch
import copy
from tqdm import tqdm
import torch.nn as nn
from pytorch_lightning import seed_everything
from torch.nn import functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

from src.data_utils import *
#from src.model_utils import setupTokenizer
from src.datasethandler import NarrationDataSet
from src.inferenceUtils import PerformanceNarrator
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback,T5Config
from src.modeling_bart import BartNarrationModel


In [2]:

from modeling_t5 import DataNarration
from SelfAttentionBasedTableEncoder import (CollapsedMetricsTableEncoder,)




In [3]:
processed = pk.load(open('dataset/train_dataset_new.dat', 'rb'))

#Load the test set
test_data = json.load(open('dataset/test set.json'))
test_sample = []
eval_tables = []
for pc in test_data:
    test_sample.append(processInputTableAndNarrations(
        pc, identical_metrics=identicals))


rtest_sample = []
reval_tables = []
for pc in test_data:
    rtest_sample.append(processInputTableAndNarrations(
        pc, identical_metrics=identicals))

In [4]:
modelbase = 't5-base'
# Process the data and set up the tokenizer
narrationdataset = NarrationDataSet(modelbase,
                                    max_preamble_len=160,
                                    max_len_trg=185, max_rate_toks=8,
                                    lower_narrations=True,
                                    process_target=True)

narrationdataset.fit(processed, test_sample)

dataset = narrationdataset.train_dataset
test_dataset = narrationdataset.test_dataset
tokenizer = tokenizer_ = narrationdataset.tokenizer_


train_dataset, val_dataset = dataset, test_dataset


train_size = int(len(dataset))
val_size = int(len(test_dataset))
print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

4,529 training samples
  100 validation samples


In [5]:
from src.model_utils import setupTokenizer

import math




In [6]:
arguments = train_arguments = {'output_dir':'TrainModels/','warmup_ratio':0.2,
'per_device_train_batch_size':8,'num_train_epochs':2,
'lr_scheduler_type':'cosine','learning_rate':5e-5,'evaluation_strategy':'steps','logging_steps':500,'seed':456}

In [7]:
# Get the model
from src.trainer_utils import getTrainingArguments,CustomTrainerFusion
def get_model(dataset, model_type):
    def getModel():
        if 'bart' in dataset.modelbase:
            return BartNarrationModel(
            vocab_size=len(dataset.tokenizer_), model_type=model_type,
            modelbase=dataset.modelbase,)
        else:
            return T5NarrationModel(
            vocab_size=len(dataset.tokenizer_), model_type=model_type,
            modelbase=dataset.modelbase,)
    return getModel


In [8]:
class CustomTrainerFusion(Trainer):
    vocab_size: int
    scale_loss: bool = False
    prev_seq_loss = []
    ce_loss = []
    total_loss =[]
    device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')

    
    
    def compute_loss(self, model, inputs, return_outputs=False):
        comp_input =  inputs
        outputs = model(comp_input,)
        
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

In [9]:
seed_everything(456)
device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')

# Build actual trainingArgument object
training_arguments = getTrainingArguments(train_arguments)

getModel = get_model(narrationdataset,model_type='earlyfusion')


trainer = CustomTrainerFusion(model_init=getModel,
                        args=training_arguments,
                        train_dataset=narrationdataset.train_dataset,
                        eval_dataset=narrationdataset.test_dataset,
                        callbacks=[EarlyStoppingCallback(early_stopping_patience=4)])

trainer.train()

Global seed set to 456
loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /home/essel/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
 

earlyfusion
Early Fusion


All model checkpoint weights were used when initializing DataNarration.

Some weights of DataNarration were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.relative_attention_for_table.mask', 'encoder.relative_attention_for_table.query.bias', 'encoder.relative_attention_for_table.value.weight', 'encoder.relative_attention_for_table.key.bias', 'encoder.relative_attention_for_table.Er', 'encoder.relative_attention_for_table.key.weight', 'encoder.relative_attention_for_table.query.weight', 'encoder.relative_attention_for_table.value.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /home/essel/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637
Model config T5Config {
  "archit

earlyfusion
Early Fusion


All model checkpoint weights were used when initializing DataNarration.

Some weights of DataNarration were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.relative_attention_for_table.mask', 'encoder.relative_attention_for_table.query.bias', 'encoder.relative_attention_for_table.value.weight', 'encoder.relative_attention_for_table.key.bias', 'encoder.relative_attention_for_table.Er', 'encoder.relative_attention_for_table.key.weight', 'encoder.relative_attention_for_table.query.weight', 'encoder.relative_attention_for_table.value.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
***** Running training *****
  Num examples = 4529
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1134
Automatic Weights & Biases logging enabled, to disable 

...

Saving model checkpoint to TrainModels/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


{'eval_loss': 1.450796365737915, 'eval_runtime': 0.6898, 'eval_samples_per_second': 144.972, 'eval_steps_per_second': 18.846, 'epoch': 1.76}


Deleting older checkpoint [TrainModels/checkpoint-500] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from TrainModels/checkpoint-1000 (score: 1.450796365737915).


{'train_runtime': 299.533, 'train_samples_per_second': 30.24, 'train_steps_per_second': 3.786, 'train_loss': 2.651685401876137, 'epoch': 2.0}


TrainOutput(global_step=1134, training_loss=2.651685401876137, metrics={'train_runtime': 299.533, 'train_samples_per_second': 30.24, 'train_steps_per_second': 3.786, 'train_loss': 2.651685401876137, 'epoch': 2.0})

In [10]:
narrator = PerformanceNarrator(trainer.model,narrationdataset,device,sampling=False,verbose=False)

In [53]:
#torch.unsqueeze??

In [64]:
test_dataset[0]['preamble_tokens'].unsqueeze(0).shape

torch.Size([1, 160])

In [14]:
example = test_sample[40]
seed = 456

In [15]:
outt=narrator.generateNarration(example, seed,  max_length=190,
                          length_penalty=8.6, beam_size=10,
                          repetition_penalty= 1.5,
                           return_top_beams=4)

Global seed set to 456


In [16]:
outt

['The model has an accuracy of 74.67%, an AUC score of 73.99%, and an F2score of 66.21%. From these scores, we can see that the model performs very well in terms of correctly predicting the true labels for a number of test cases belonging to any of the class labels. Furthermore, the specificity score achieved is equal to 84.17%. These scores indicate that this model will not be able to correctly identify the correct label for the majority of all test instances.']

In [76]:
outt

['The performance of the model on this binary classification problem (where a given test observation is labeled as either #CA or #CB or #CC ) was evaluated based on the metrics: recall, AUC, accuracy, precision, and recall. The scores achieved by the classifier are very high, indicating that it will be effective in terms of its prediction decisions for several test cases with only a small margin of error.']